In [ ]:
#Importing libraries
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#importing the json file
#data_file=open(data_root+'/intents.json').read()
#data=json.loads(data_file)
with open('intents.json', 'r') as file:
    data = json.load(file)

#the dataset has objects known as intents
#intents are having their own tags showing context,patterns and responese
#these intents will help us to reply to those statements.

In [ ]:
#Creating data_X and data_Y
#using NLTK's WordNetLemmatizer(). This function will help
#us to reduce the words and convert it into their root words

words=[] #For Row model/ vocabulary for patterns
classes=[] #For Row model/vocalbulary for tags
data_X=[] #For storing each pattern
data_Y=[] #For storing tag corresponding to each pattern in data_X

#Iterating all intents in intents.json

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens=nltk.word_tokenize(pattern) #tokenizing each pattern
    words.extend(tokens) #append tokens into words
    data_X.append(pattern) #appending pattern to data_X
    data_Y.append(intent["tag"]) #appending the associated tag to each pattern

  if intent["tag"] not in classes: #appending the tag if it is not present
    classes.append(intent["tag"])

lemmatizer=WordNetLemmatizer()
#lemmatizer to get the root of words
#turning everything into lowercase and removing all punctuations

words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
#Sorting the vocabulary and classes into aplhabetical order
#checking if there are no duplicate orders

words=sorted(set(words))
classes=sorted(set(classes))


In [ ]:
#Data TRAINING

#We are converting our text into numbers using BOW model(Bag of words)
#words and classes act as a vocabulary for patterns and tags respectively

#1 if the word is present in the pattern/tag being read(from data_X)
#0 if absent.

#Data will be converted into numbers
#stored into two arrays: train_X and train_Y respectively.


training=[]
out_empty=[0]*len(classes) #intializing

#BAG of model
for idx,doc in enumerate(data_X):
  bow=[]
  text=lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)


  output_row=list(out_empty)
  output_row[classes.index(data_Y[idx])]=1

  #add the one hot encoded BOW and associated classes to traininhg

  training.append([bow, output_row])

#shuffle the data and convert it to an array

random.shuffle(training)
training=np.array(training,dtype=object)

#spliting the features and target_labels
train_X=np.array(list(training[:,0]))
train_Y=np.array(list(training[:,1]))

In [ ]:
#Building a neural network

#I am using Keras sequential model
#input of this network will be the array train_X.
#3 layers:
#1st layer:128 neurons
#2nd layer: 64 neurons
#last layer: the same number of neurons as the length of one element of train_Y

#Training model for about 150 times


#We have used a Dropout layer

model=Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]),activation="softmax"))
adam=  tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
#adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)

model.compile(loss='categorical_crossentropy',
               optimizer=adam,
               metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X,y=train_Y, epochs=150,verbose=1)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 128)                 │          10,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 19)                  │           1,235 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,243 (79.07 KB)

 Trainable params: 20,243 (79.07 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0593 - loss: 2.9430
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1418 - loss: 2.8691 
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1106 - loss: 2.7585 
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1442 - loss: 2.5407 
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3381 - loss: 2.2633 
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3686 - loss: 2.1249 
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4151 - loss: 1.9843 
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4896 - loss: 1.9188
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5697 - loss: 1.5932 
Epoch 10/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5513 - loss: 1.5394 
Epoch 11/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6731 - loss: 1.3234 
Epoch 12/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7220 

In [ ]:
#now i will start to create a chatbot
#We will create few easy functions that will convert the user's input query to arrays and predict the relevant tag for it.


#Preprocessing of the input

def clean_text(text):
  tokens=nltk.word_tokenize(text)
  tokens=(lemmatizer.lemmatize(word) for word in tokens)
  return tokens

def bag_of_words(text,vocab):
  tokens=clean_text(text)
  bow=[0]*len(vocab)
  for w in tokens:
    for idx,word in enumerate(vocab):
      if word==w:
        bow[idx]=1
  return np.array(bow)

def pred_class(text,vocab,labels):
  bow=bag_of_words(text, vocab)
  result=model.predict(np.array([bow]))[0] #extracting proabilities

  thresh=0.5
  y_pred=[[indx, res] for indx, res in enumerate(result) if res>thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  #Sorting by values of probability in decresing order
  return_list=[]

  for r in y_pred:
    return_list.append(labels[r[0]])
    #contains labels(tags) for highest proability
  return return_list

def get_response(intents_list, intents_json):
  if len(intents_list)==0:
    result="Sorry! , I didn't understand."
  else:
    tag=intents_list[0]
    list_of_intents=intents_json["intents"]
    for i in list_of_intents:
      if i["tag"]==tag:
        result=random.choice(i["responses"])
        break
  return result


In [ ]:
print("Press 0 if you don't want to chat with our ChatBot.")
while True:
  message=input("")
  if message =="0":
    break
  intents=pred_class(message,words,classes)
  result=get_response(intents,data)
  print(result)

Press 0 if you don't want to chat with our ChatBot.
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Hey!
please suggest me a course
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
You could look into courses on project management or agile methodologies if you're interested in managing projects.
i want a project on sentiment analysis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Sure, I will let our team know. You will get email about this sonn on your registered email id.
suggest me a project
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
You could build a simple weather forecasting app.
0
